# Plant Disease Image Recognition Anaylsis

# Imports

In [1]:
import os

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Constants

In [ ]:
PATH = "/Users/maukanmir/Downloads/archive/"
train_path, test_path, val_path = PATH + "Train/", PATH + "Test/", PATH +"Validation/"

# Functions

# Load Data